In [151]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot  as plt
%matplotlib inline

In [258]:
path = os.getcwd() 
data_root = path + '\\DATA\\'
data_root

'C:\\Users\\Kaipable\\Desktop\\GitHub\\voter-protection-corps\\pollworkers_src\\DATA\\'

In [259]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [284]:
#Read In
all_df = pd.read_csv('all_states_risk_data.csv')

### Customize Metrics and  Indicators
Define Your Categories, Variables, and Abbreviations Below in metric_map

In [261]:
metric_map = {
    'priority population' : ('PRI',['Non-white Population','Senior Residents']),
    'demand' : ('D',['In-Person Voters', 'Inactive Voters']),
    'supply' : ('S',['Voters Per Location','Voters Per Machine','Voters Per Pollworker','Wait Times']),
    'poll worker recruiting' : ('REC',['Senior Poll Workers','Reported Difficulty '])
}
the_metrics = [i for i in metric_map.keys()]
#the_indicators = [ for k in the_metrics for acronym, indicator_list in metric_map[k]

agg_metrics = ('mean', 'median', 'max', 'min')

In [387]:
# Get Percentile Columns
def get_percentiles(df):
    for i in metric_map.keys():
        for j in metric_map[i][1]:
            keep_series=df.groupby("State_Abbr")[j].count() >0
            viable_states = keep_series[keep_series==True].index
            temp_df = df[df['State_Abbr'].isin(viable_states)]
            intermed= temp_df.groupby("State_Abbr")[j].rank(pct=True, na_option="keep")
            df["("+metric_map[i][0]+") "+j] = intermed
    return df
        
def get_agg_columns(df):
    funct_map = {'median': np.median,
            'mean': np.mean, 
            'min': np.min,
            'max': np.max}
    for metric in metric_map.keys():
        percent_list = ["("+metric_map[metric][0]+") " + j for j in metric_map[metric][1]] 
        df[metric.title()+" ("+agg_method.title()+")"] = funct_map[agg_method](df[percent_list], axis=1)
        if agg_method not in funct_map.keys():
            print('Please select an acceptable metric: "min", "max", "mean", or "median".')
    return df

def get_final_scores(df):
    for i in metric_map.keys():
        keep_series=here_df.groupby("State_Abbr")[i.title()+" ("+agg_method.title()+")"].count() >0
        viable_states = keep_series[keep_series==True].index
        temp_df = here_df[df['State_Abbr'].isin(viable_states)]
        rel_scores = temp_df.groupby("State_Abbr")[i.title()+" ("+agg_method.title()+")"].rank(pct=True, na_option="keep")
        rel_heat = rel_scores.apply(lambda x: "High" if x > 0.75  else ("Low" if x <0.25 else (np.nan if np.isnan(x) else "Medium")))
        here_df[i.title()+" (Final)"] = rel_scores
        here_df[i.title()+" (LMH)"]= rel_heat
    return df

def get_final_df(df, agg_method, metric_map):
    func_list = [get_percentiles, get_agg_columns, get_final_scores]
    for i in func_list:
        df = i(df)
    return final_df

In [388]:
final_df = get_final_df(all_df, 'median', metric_map)

In [390]:
final_df.head()

,Unnamed: 0,Full State,short_fips,State_Abbr,County Name,Non-white Population,Senior Residents,Senior Poll Workers,Reported Difficulty,In-Person Voters,Inactive Voters,Voters Per Location,Voters Per Machine,Voters Per Pollworker,Wait Times,Wait Count,# of Same Day Locations,# of Machines,# of Poll Workers,# of Senior Poll Workers,# of Voters Removed,# of Precincts,# Mail In Ballots,% Early Vote,% Rejected Ballots,Median Age,Median Household Income,Total Population,Total Eligible Voters,% Senior Poll Worker,% Votes in Person,% Inactive Voters,%Nonwhite,% Residents Over 60,(PRI) Non-white Population,(PRI) Senior Residents,(D) In-Person Voters,(D) Inactive Voters,(S) Voters Per Location,(S) Voters Per Machine,(S) Voters Per Pollworker,(S) Wait Times,(REC) Senior Poll Workers,(REC) Reported Difficulty,Priority Population (Median),Demand (Median),Supply (Median),Poll Worker Recruiting (Median),Priority Population (Final),Priority Population (LMH),Demand (Final),Demand (LMH),Supply (Final),Supply (LMH),Poll Worker Recruiting (Final),Poll Worker Recruiting (LMH)
0,0,ALABAMA,1001,AL,AUTAUGA COUNTY,12763.0,10744.0,96.0,4.0,19081.0,3379.0,2093.157895,864.565217,337.033898,0.0,0.0,19.0,46.0,118.0,96.0,2678.0,19.0,728.0,0.0,0.005195,37.8,58786.0,55200.0,39770.0,0.813559,0.960147,0.084964,0.231214,0.194638,0.656716,0.597015,0.656716,0.686567,0.805970,0.791045,0.910448,0.5,0.477612,0.567164,0.626866,0.671642,0.798507,0.522388,0.671642,Medium,0.679104,Medium,0.865672,High,0.507463,Medium
1,1,ALABAMA,1003,AL,BALDWIN COUNTY,28581.0,55516.0,424.0,3.0,75068.0,13826.0,3165.857143,1251.024194,300.052224,0.0,0.0,49.0,124.0,517.0,424.0,14145.0,91.0,2748.0,0.0,0.007122,42.8,55962.0,208107.0,155127.0,0.820116,0.959164,0.089127,0.137338,0.266767,0.820896,0.955224,0.940299,0.955224,0.940299,0.955224,0.865672,0.5,0.940299,0.231343,0.888060,0.947761,0.902985,0.585821,0.895522,High,0.955224,High,0.970149,High,0.656716,Medium
2,2,ALABAMA,1005,AL,BARBOUR COUNTY,13566.0,6285.0,72.0,4.0,7690.0,1452.0,184.287234,524.939394,190.362637,0.0,0.0,94.0,33.0,91.0,72.0,1409.0,21.0,684.0,0.0,0.095662,39.9,34186.0,25782.0,17323.0,0.791209,0.909521,0.083819,0.526181,0.243775,0.686567,0.373134,0.343284,0.373134,0.029851,0.597015,0.597015,0.5,0.343284,0.567164,0.529851,0.358209,0.548507,0.455224,0.567164,Medium,0.373134,Medium,0.537313,Medium,0.417910,Medium
3,3,ALABAMA,1007,AL,BIBB COUNTY,5259.0,4877.0,52.0,3.0,6668.0,999.0,1780.500000,749.684211,200.619718,0.0,0.0,8.0,19.0,71.0,52.0,886.0,16.0,196.0,0.0,0.000000,39.9,45340.0,22527.0,14244.0,0.732394,0.970597,0.070135,0.233453,0.216496,0.253731,0.283582,0.238806,0.238806,0.716418,0.716418,0.626866,0.5,0.231343,0.231343,0.268657,0.238806,0.671642,0.231343,0.268657,Medium,0.238806,Low,0.716418,Medium,0.104478,Low
4,4,ALABAMA,1009,AL,BLOUNT COUNTY,2591.0,13846.0,104.0,4.0,19120.0,2335.0,1526.423077,763.211538,317.496000,0.0,0.0,26.0,52.0,125.0,104.0,2350.0,29.0,426.0,0.0,0.002232,40.8,48695.0,57645.0,39687.0,0.832000,0.977405,0.058835,0.044948,0.240194,0.119403,0.671642,0.671642,0.507463,0.656716,0.731343,0.895522,0.5,0.537313,0.567164,0.395522,0.589552,0.694030,0.552239,0.388060,Medium,0.574627,Medium,0.746269,Medium,0.559701,Medium
